In [48]:
import pandas as pd
import numpy as np
import json
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [49]:
file = open(r"C:\\DISK_D\\Personal\\ASL\\archive\\WLASL_v0.3.json")
data = json.load(file)
len(data)

2000

In [82]:
total_video = 0
train = []
test = []
val = []
total_data = []

In [83]:
for i in range(len(data)):
    gloss = data[i]["gloss"]
    instances = data[i]["instances"]
    #print(len(instances),i)
    total_video += len(instances)
    for instance in instances:
        total_data.append([instance["video_id"]+".mp4",gloss,instance["split"]])
        if instance["split"] == "train":
            video_id = instance["video_id"]
            split = instance["split"]
            bboxes = instance["bbox"]
            train.append({"video_id":video_id+".mp4","gloss":gloss,"bbox":bboxes})
        if instance["split"] == "test":
            video_id = instance["video_id"]
            split = instance["split"]
            bboxes = instance["bbox"]
            test.append({"video_id":video_id+".mp4","gloss":gloss,"bbox":bboxes})
        if instance["split"] == "val":
            video_id = instance["video_id"]
            split = instance["split"]
            bboxes = instance["bbox"]
            val.append({"video_id":video_id+".mp4","gloss":gloss,"bbox":bboxes})



In [84]:
train[:2]

[{'video_id': '69241.mp4', 'gloss': 'book', 'bbox': [385, 37, 885, 720]},
 {'video_id': '65225.mp4', 'gloss': 'book', 'bbox': [190, 25, 489, 370]}]

In [85]:
total_data[:5]

[['69241.mp4', 'book', 'train'],
 ['65225.mp4', 'book', 'train'],
 ['68011.mp4', 'book', 'train'],
 ['68208.mp4', 'book', 'train'],
 ['68012.mp4', 'book', 'train']]

In [39]:
len(train),len(test),len(val),len(train)+len(test)+len(val),total_video

(14289, 2878, 3916, 21083, 21083)

In [63]:
total_data[0][0]

'69241.mp4'

In [86]:
absent_video_ids = []
for item in total_data:
    vid_path = os.path.join(r"C:\\DISK_D\\Personal\\ASL\\archive\\videos",item[0])
    if os.path.exists(vid_path):
        pass
    else:
        #print(item["video_id"], " Doesn't exist")
        absent_video_ids.append(item[0])

len(absent_video_ids),absent_video_ids[:5]

(9103, ['65225.mp4', '68011.mp4', '68208.mp4', '68012.mp4', '70212.mp4'])

In [87]:
# Create a set from list2 for faster lookup
set_list2 = set(absent_video_ids)

# Remove items from list1 if the first element is present in set_list2
new_list1 = [item for item in total_data if item[0] not in set_list2]

print(len(new_list1),len(total_data))

11980 21083


In [89]:
video_df = pd.DataFrame(new_list1,columns=["video_id","gloss","split"])
video_df.head()

,video_id,gloss,split
0,69241.mp4,book,train
1,07069.mp4,book,train
2,07068.mp4,book,train
3,07070.mp4,book,train
4,07099.mp4,book,val


In [110]:
unique_values = video_df["gloss"].value_counts()
less_video_gloss_list = unique_values[unique_values==3].index.tolist()
less_video_data = video_df[video_df["gloss"].isin(less_video_gloss_list)]
less_video_data

,video_id,gloss,split
5440,24057.mp4,garage,train
5441,24059.mp4,garage,train
5442,24063.mp4,garage,val
6869,41164.mp4,parents,train
6870,41165.mp4,parents,val
...,...,...,...
11971,62783.mp4,weigh,train
11972,62785.mp4,weigh,train
11977,63186.mp4,whistle,train
11978,63188.mp4,whistle,train


In [ ]:
train_absent_video_ids = []
for item in train:
    vid_path = os.path.join(r"C:\\DISK_D\\Personal\\ASL\\archive\\videos",item["video_id"])
    if os.path.exists(vid_path):
        pass
    else:
        print(item["video_id"], " Doesn't exist")
        train_absent_video_ids.append(item["video_id"])

test_absent_video_ids = []
for item in test:
    vid_path = os.path.join(r"C:\\DISK_D\\Personal\\ASL\\archive\\videos",item["video_id"])
    if os.path.exists(vid_path):
        pass
    else:
        print(item["video_id"], " Doesn't exist")
        test_absent_video_ids.append(item["video_id"])

val_absent_video_ids = []
for item in val:
    vid_path = os.path.join(r"C:\\DISK_D\\Personal\\ASL\\archive\\videos",item["video_id"])
    if os.path.exists(vid_path):
        pass
    else:
        print(item["video_id"], " Doesn't exist")
        val_absent_video_ids.append(item["video_id"])

In [44]:
len(train_absent_video_ids),len(test_absent_video_ids),len(val_absent_video_ids)

(5976, 1464, 1663)

In [46]:
img_id = 500
video_path = os.path.join(r"C:\\DISK_D\\Personal\\ASL\\archive\\videos",train[img_id]["video_id"])
bbox = train[img_id]["bbox"]
bbox
if os.path.exists(video_path):
    print("File exists")
else:
    print("File doesn't exist")

File exists


In [47]:
cap = cv2.VideoCapture(video_path)
if cap.isOpened():
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Draw bbox
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)
            # Display the frame
            cv2.imshow('Video', frame)

            # Press 'q' to exit the loop
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

cap.release()
cv2.destroyAllWindows()